In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
train_data = train_datagen.flow_from_directory(r'C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\mrlEyes_2018_01\Prepared_Data\train', target_size=(80,80), batch_size=batchsize, class_mode='categorical', subset='training')

validation_data = train_datagen.flow_from_directory(r'C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\mrlEyes_2018_01\Prepared_Data\train', target_size=(80,80), batch_size=batchsize, class_mode='categorical', subset='validation')

Found 61519 images belonging to 2 classes.
Found 15379 images belonging to 2 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\mrlEyes_2018_01\Prepared_Data\test', target_size=(80,80), batch_size=8, class_mode='categorical')


Found 8000 images belonging to 2 classes.


In [14]:
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))


In [15]:
hmodel = base_model.output

In [16]:
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D

# Remove the existing custom layers
# Adjust the following lines based on your MobileNetV2-based model

hmodel = GlobalAveragePooling2D()(hmodel)  # Use GlobalAveragePooling2D for MobileNetV2
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)  # Assuming 2 output classes

# Create the drowsiness detection model
model = Model(inputs=base_model.input, outputs=hmodel)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with the appropriate optimizer, loss, and metrics
model.compile(
    optimizer=Adam(lr=0.001),  # Adjust the learning rate as needed
    loss='categorical_crossentropy',  # Change the loss function based on your task
    metrics=['accuracy']
)


In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [21]:
checkpoint = ModelCheckpoint(r'C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\Model2\model.h5', monitor='val_loss', save_best_only = True, verbose=3)
2
earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor = 'val_loss', patience=3, verbose=3)

callbacks = [checkpoint, earlystop, learning_rate]

In [22]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(
    train_data,
    validation_data=validation_data,
    validation_steps=validation_data.samples // batchsize,
    callbacks=callbacks,
    epochs=50
)

Epoch 1/50
7689/7690 [============================>.] - ETA: 0s - loss: 0.2459 - accuracy: 0.9021
Epoch 1: val_loss improved from inf to 0.32961, saving model to C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\Model2\model.h5


C:\Users\Avinash\miniconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


7690/7690 [==============================] - 503s 64ms/step - loss: 0.2459 - accuracy: 0.9020 - val_loss: 0.3296 - val_accuracy: 0.8408 - lr: 0.0010
Epoch 2/50
7689/7690 [============================>.] - ETA: 0s - loss: 0.2021 - accuracy: 0.9203
Epoch 2: val_loss improved from 0.32961 to 0.27870, saving model to C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\Model2\model.h5
7690/7690 [==============================] - 492s 64ms/step - loss: 0.2021 - accuracy: 0.9203 - val_loss: 0.2787 - val_accuracy: 0.8802 - lr: 0.0010
Epoch 3/50
7690/7690 [==============================] - ETA: 0s - loss: 0.1919 - accuracy: 0.9232
Epoch 3: val_loss did not improve from 0.27870
7690/7690 [==============================] - 486s 63ms/step - loss: 0.1919 - accuracy: 0.9232 - val_loss: 0.3568 - val_accuracy: 0.8550 - lr: 0.0010
Epoch 4/50
7689/7690 [============================>.] - ETA: 0s - loss: 0.1821 - accuracy: 0.9272
Epoch 4: val_loss did not improve from 0.27870
7690/7690 [==========

# Model Evaluation

In [4]:
from tensorflow.keras.models import load_model
saved_model = load_model(r"C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\Model2\model.h5")


In [5]:
test_accuracy = saved_model.evaluate(test_data)[1]
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

1000/1000 [==============================] - 28s 27ms/step - loss: 0.3319 - accuracy: 0.8553
Test Accuracy: 85.53%


In [8]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report  # Import confusion_matrix

# Load the saved model
saved_model = load_model(r"C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\Model2\model.h5")

# Define the test data generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(
    r'C:\Users\Avinash\Desktop\Major_Project\Drowsiness_detection\mrlEyes_2018_01\Prepared_Data\test',
    target_size=(80, 80),
    batch_size=8,
    class_mode='categorical',
    shuffle=False  # Set shuffle to False to maintain order for the labels
)

# Use the model to make predictions on the test data
y_pred = saved_model.predict(test_data)

# Obtain the true labels from the directory structure
# The class labels are inferred from the subdirectories in your test directory
# You can use the class_indices attribute to map class labels to their corresponding subdirectories
true_labels = (test_data.class_indices.keys())

# Calculate confusion matrix and classification report
# Assuming you have true labels as subdirectories in your test directory
confusion = confusion_matrix(test_data.classes, y_pred.argmax(axis=1))
classification_rep = classification_report(test_data.classes, y_pred.argmax(axis=1), target_names=true_labels)

print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", classification_rep)


Found 8000 images belonging to 2 classes.
1000/1000 [==============================] - 27s 26ms/step
Confusion Matrix:
 [[3902   98]
 [1060 2940]]
Classification Report:
               precision    recall  f1-score   support

  Close_Eyes       0.79      0.98      0.87      4000
   Open_Eyes       0.97      0.73      0.84      4000

    accuracy                           0.86      8000
   macro avg       0.88      0.86      0.85      8000
weighted avg       0.88      0.86      0.85      8000

